# Evaluation v2

In [1]:
from picturedata.generator import DataGenerator
from keras.models import load_model
from keras.preprocessing import image, sequence
import numpy as np
import pandas as pd

Using TensorFlow backend.


In [2]:
generator = DataGenerator()

Total samples : 383454
Vocabulary size: 8256
Maximum caption length: 40
Image count: 30000


In [13]:
model = load_model('Models/WholeModel.h5')

In [4]:
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_2 (InputLayer)             (None, 40)            0                                            
____________________________________________________________________________________________________
input_1 (InputLayer)             (None, 1000)          0                                            
____________________________________________________________________________________________________
embedding_1 (Embedding)          (None, 40, 256)       2113536     input_2[0][0]                    
____________________________________________________________________________________________________
dense_1 (Dense)                  (None, 128)           128128      input_1[0][0]                    
___________________________________________________________________________________________

In [20]:
image_feature = generator.images[1]
start = [generator.word_index['<start>']]
captions = [[start, 0.0]]
while(len(captions[0][0]) < generator.max_cap_len):
    temp_captions = []
    for caption in captions:
        partial_caption = sequence.pad_sequences([caption[0]], maxlen=generator.max_cap_len, padding='post')
        next_words_pred = model.predict([np.asarray([image_feature]), np.asarray(partial_caption)])[0]
        next_words = np.argsort(next_words_pred)[-3:]
        for word in next_words:
            new_partial_caption, new_partial_caption_prob = caption[0][:], caption[1]
            new_partial_caption.append(word)
            new_partial_caption_prob+=next_words_pred[word]
            temp_captions.append([new_partial_caption,new_partial_caption_prob])
    
    captions = temp_captions
    captions.sort(key = lambda l:l[1])
    captions = captions[-3:]

    
captions.sort(key = lambda l:l[1])
best_caption = captions[-1][0]
caption = " ".join([generator.index_word[index] for index in best_caption])

caption_split = caption.split()
processed_caption = caption_split[1:]

try:
    end_index = processed_caption.index('<end>')
    processed_caption = processed_caption[:end_index]
except:
    pass
sentence = " ".join([word for word in processed_caption])
print sentence


dog is jumping on a field .


In [22]:
import nltk

In [26]:
f_captions = open('Flickr8k/Flickr8k_text/Flickr8k.token.txt', 'rb')
captions_text = f_captions.read().strip().split('\n')
image_captions_pair = {}
for row in captions_text:
    row = row.split("\t")
    row[0] = row[0][:len(row[0])-2]
    try:
        image_captions_pair[row[0]].append(row[1])
    except:
        image_captions_pair[row[0]] = [row[1]]
    f_captions.close()

hypotheses=[]
references = []
hypothesis = sentence
reference = image_captions_pair[generator.data['image'][0]]
hypotheses.append(hypothesis)
references.append(reference)

In [27]:
print nltk.translate.bleu_score.corpus_bleu(references, hypotheses)

0.320570849039
